In [ ]:
# %load MNIST_CNN.py
"""
Created on Sun Nov 17 21:00:47 2019

@author: Carboxy
"""

import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "CPU")
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1=nn.Conv2d(1,10,5) #24*24,10
        self.conv2=nn.Conv2d(10,20,3) #10*10,128
        self.fc1=nn.Linear(20*10*10,400)
        self.fc2=nn.Linear(400,10)
    def forward(self,x):
        input_size=x.size(0)
        out=self.conv1(x) #24*24
        out=torch.nn.functional.relu(out)
        out=torch.nn.functional.max_pool2d(out,2,2) #12
        out=self.conv2(out) #10*10
        out=torch.nn.functional.relu(out)
        out=out.view(input_size,-1)
        out=self.fc1(out)
        out=torch.nn.functional.relu(out)
        out=self.fc2(out)
        out=torch.nn.functional.log_softmax(out,dim=1)
        return out


    
model = SimpleNet().to(DEVICE)

# TODO:define loss function and optimiter
#criterion = torch.nn.functional.nll_loss(output,target)
optimizer = torch.optim.Adam(model.parameters())

# train and evaluate
for epoch in range(NUM_EPOCHS):
    for batch_idx,(images, labels) in enumerate(train_loader):
        # TODO:forward + backward + optimize
        images,labels=images.to(DEVICE),labels.to(DEVICE)
        optimizer.zero_grad()
        output=model(images)
        criterion = torch.nn.functional.nll_loss(output,labels)
        criterion.backward()
        optimizer.step()
        if(batch_idx%50==0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(images), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), criterion.item()))
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset

correct = 0
with torch.no_grad():
    for batch_idx,(images, labels) in enumerate(train_loader):
        images,labels=images.to(DEVICE),labels.to(DEVICE)
        output = model(images)
        pred = output.max(1,keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
print('\nTrain Accuracy: {}/{} ({:.2f}%)'.format(correct,len(
                train_loader.dataset),100.*correct/
                len(train_loader.dataset)))
                
correct = 0
with torch.no_grad():
    for batch_idx,(images, labels) in enumerate(test_loader):
        images,labels=images.to(DEVICE),labels.to(DEVICE)
        output = model(images)
        pred = output.max(1,keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
print('\nTest Accuracy: {}/{} ({:.2f}%)'.format(correct,len(
                test_loader.dataset),100.*correct/
                len(test_loader.dataset)))